In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

# 1. Raw Data Loading
df = pd.read_csv('./data/titanic/train.csv')
display(df.head())
train = df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
# 2. feature engineering
# 데이터를 숫자 형태로 바꾸어, 합치거나 추출할 feature 추출

# PassengerID : 승객일련번호 => 필요X

# Survived : 생존여부, 0 사망, 1 생존
#            label 필요 

# Pclass : 객실 등급 (1등급, 2등급, 3등급)
#          상관계수를 구해보고 연관성을 파악한 후 살리는게 좋을 것 같다.

# Name : 필요 x
#        나이 결측치 결정 시 활용

# Age : 나이. 연관성이 있어서 필요
#       결측치 존재
#       결측치를 채워주는 가장 쉬운 방법은, 모든 사람의 나이 평균
#       그 사람 이름에 포함된 MR,MRS, MISS를 따져서 그 분류의 평균 나이 사용
#       유아 0, 청소년 1, 성인 2, 노인 3

# Sex : male, female => 0, 1로 변경

# SibSp, Parch :  두 컬럼을 합쳐서 num_of_family로 만들어 사용

# Ticket : 삭제

# Fare : 요금, 고민을 해 보아야 

# cabin : 객실번호, 필요 X

# Embarked : 어디에서 승선했는지. S, C, Q => 숫자로 바꾸어서 처리

# pandas를 이용하여 학습에 적합한 형태로 데이터를 처리

In [5]:
# 3. Data Preprocessing


# 필요없는 column부터 제거
train.drop(['PassengerId', 'Name', 'Ticket','Fare','Cabin'], axis=1, inplace=True)
# display(train)

# 성별처리
sex_mapping = {'male': 0, 'female': 1}
train['Sex'] = train['Sex'].map(sex_mapping)

# display(train)

# 가족처리
train['Family'] = train['SibSp'] + train['Parch']
train.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# print(train.isnull().sum())

# Embarked 결측치 처리
train['Embarked'] = train['Embarked'].fillna('Q')
# print(train.isnull().sum())

# Age에 대한 결측치 처리
train['Age'] = train['Age'].fillna(train['Age'].mean())
# print(train.isnull().sum())

# Embarked S, C, Q => 숫자로 바꾸어서 처리
embarked_mapping = {'S': 0, 'C':1, 'Q':2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
# display(train)

# Age에 대해서 Binning 처리 (Numerical value => categorical value)
train.loc[train['Age'] < 8, 'Age'] = 0 # loc[행, 렬]
train.loc[(train['Age'] >= 8) & (train['Age'] < 20), 'Age'] = 1
train.loc[(train['Age'] >= 20) & (train['Age'] < 65), 'Age'] = 2
train.loc[train['Age'] >= 65, 'Age'] = 3
display(train)

,Survived,Pclass,Sex,Age,Embarked,Family
0,0,3,0,2.0,0,1
1,1,1,1,2.0,1,1
2,1,3,1,2.0,0,0
3,1,1,1,2.0,0,1
4,0,3,0,2.0,0,0
...,...,...,...,...,...,...
886,0,2,0,2.0,0,0
887,1,1,1,1.0,0,0
888,0,3,1,2.0,0,3
889,1,1,0,2.0,1,0


- 데이터가 준비되었다.
- Tensorflow를 이용해서 학습을 진행하고 Accuracy를 구현하자.
- 891개의 데이터를 다 학습에 사용하지 않고, 7:3비율로 나누어 학습과 정확도 측정에 사용한다.

In [6]:
# train용 데이터, test용 데이터를 분리한다.

train_data = train.iloc[:int(train.shape[0] * 0.7)]
test_data = train.iloc[int(train.shape[0] * 0.7):]

train_x_data = train_data.drop('Survived', axis=1, inplace=False).values
train_t_data = train_data['Survived'].values.reshape(-1, 1)

test_x_data = test_data.drop('Survived', axis=1, inplace=False).values
test_t_data = test_data['Survived'].values.reshape(-1, 1)

# placeholder
X = tf.placeholder(shape=[None, 5], dtype=tf.float32) # 독립변수의 데이터를 받기 위한 placeholder
T = tf.placeholder(shape=[None, 1], dtype=tf.float32) # 종속변수 label의 데이터를 받기 위한 placeholder

# Weight & bias
W = tf.Variable(tf.random.normal([5,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# hypothesis
logit = tf.matmul(X, W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(loss)

# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(300000):
    _, W_val, b_val, loss_val = sess.run([train, W, b, loss], 
                                         feed_dict={X:train_x_data, T:train_t_data})
    
    if step % 30000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))

W : [[ 0.17050046]
 [ 0.81132567]
 [ 0.7564432 ]
 [ 0.8936088 ]
 [-0.4352367 ]], b : [1.6109718], loss : 2.146121025085449
W : [[-0.20947893]
 [ 0.79266834]
 [ 0.46455714]
 [ 0.84119797]
 [-0.51274765]], b : [1.4602337], loss : 1.2700892686843872
W : [[-0.4762455 ]
 [ 0.7874659 ]
 [ 0.24794875]
 [ 0.7971173 ]
 [-0.5305827 ]], b : [1.3539431], loss : 0.8227993845939636
W : [[-0.62744486]
 [ 0.7954996 ]
 [ 0.11385196]
 [ 0.7645423 ]
 [-0.51269776]], b : [1.2930927], loss : 0.6650026440620422
W : [[-0.70688343]
 [ 0.8120212 ]
 [ 0.03319236]
 [ 0.74137783]
 [-0.47883275]], b : [1.2607325], loss : 0.6109519004821777
W : [[-0.7487228 ]
 [ 0.83284104]
 [-0.01851294]
 [ 0.724056  ]
 [-0.4388281 ]], b : [1.2435204], loss : 0.5870900750160217
W : [[-0.77108943]
 [ 0.85595703]
 [-0.0546912 ]
 [ 0.710459  ]
 [-0.3977009 ]], b : [1.2344525], loss : 0.5726102590560913
W : [[-0.7831907 ]
 [ 0.87920284]
 [-0.08227921]
 [ 0.6988117 ]
 [-0.35713947]], b : [1.2294436], loss : 0.5616934895515442
W : [[-0.

In [ ]:
# 정확도 (Accuracy) 측정
predict = tf.cast(H >= 0.5, dtype=tf.float32) # True -> 1.0, False -> 0
correct = tf.equal(predict, T)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

accuracy_val = sess.run(accuracy, feed_dict={X:test_x_data, T:test_t_data})
print('Accuracy : {}'.format(accuracy_val))